In [1]:
import argparse
import yaml
from pathlib import Path
from typing import Dict, Any
import re
from tqdm import tqdm 
from collections import Counter
from typing import List

import pandas as pd
import torch
from transformers import AutoTokenizer

import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..')) 
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data.preprocessor import parse_problems_column, add_choices_len
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.training.model_loader import ModelConfig, load_model_inference

In [2]:
def create_configs(cfg_dict: Dict[str, Any]) -> tuple:
    model_cfg_dict = cfg_dict["model"].copy()
    model_cfg_dict["use_gradient_checkpointing"] = False
    model_cfg = ModelConfig(**model_cfg_dict)
    
    prompt_dict = cfg_dict["inference"]["prompt"]
    prompt_cfg = PromptConfig(
        policy=prompt_dict["policy"],
        mode="test",
        verbose=False
    )
    
    inference_cfg = cfg_dict.get("inference", {})
    
    return model_cfg, prompt_cfg, inference_cfg

In [3]:
with open("../../config.yaml", "r") as f:
        cfg_dict = yaml.safe_load(f)

In [4]:
model_cfg, prompt_cfg, inference_cfg = create_configs(cfg_dict)

In [5]:
adapter_path = "../../outputs/reading/final_model"
output_path = "./predictions.csv"
test_data_path = "../../data/test.csv"
max_new_tokens = inference_cfg.get("max_new_tokens", 100)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}\n")

Device: cuda



In [7]:
def load_test_data(test_path: Path) -> pd.DataFrame:
    """Load and preprocess test data."""
    test_df = pd.read_csv(test_path)
    test_df = parse_problems_column(test_df)
    test_df = add_choices_len(test_df)
    return test_df

In [8]:
print(f"Loading test data from {test_data_path}...")
test_df = load_test_data(test_data_path)
print(f"Loaded {len(test_df)} rows\n")

Loading test data from ../../data/test.csv...
Loaded 869 rows



In [9]:
test_df['paragraph'].str.len().describe()


count     869.000000
mean      887.592635
std       720.583733
min        21.000000
25%       312.000000
50%       687.000000
75%      1252.000000
max      2847.000000
Name: paragraph, dtype: float64

In [10]:
print(f"Loading tokenizer from {model_cfg.model_name_or_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_cfg.model_name_or_path,
    trust_remote_code=model_cfg.trust_remote_code,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token   

Loading tokenizer from Qwen/Qwen3-14B...


In [11]:
print(f"Loading model from {adapter_path}...")
model = load_model_inference(model_cfg, "../../models/qwen3_14B_eng_aug2/final_model_from_serverA")
model.eval()
print("Model loaded successfully!\n")

Loading model from ../../outputs/reading/final_model...
Loading Base Model for Inference: Qwen/Qwen3-14B


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading LoRA Adapter from: ../../models/qwen3_14B_eng_aug2/final_model_from_serverA
Model loaded successfully!



In [12]:
builder = PromptBuilder(prompt_cfg)
print("PromptBuilder ready!\n")

PromptBuilder ready!



In [13]:
def extract_answer(text: str) -> str:
    numbers = re.findall(r'[1-5]', text)
    
    if numbers:
        return numbers[-1]
        
    return "no"

In [14]:
def is_logic_type(text):
        calc_patterns = r"(\d+(\.\d+)?%|\d+배|\d+원|\d+달러|계산|합계|평균|비율|변화율)"
        logic_keywords = [
            "GDP", "CPI", "물가", "소득", "금리", "수요", "공급", "탄력성",
            "비용", "이윤", "이익", "상승", "하락", "증가", "감소", "환율",
            "명목", "실질", "인플레이션", "시장 구조", "독점", "과점", "진단", "증상"
        ]
        if re.search(calc_patterns, text) and any(keyword in text for keyword in logic_keywords): return True
        return False

In [15]:
test_df['is_logic'] = test_df['paragraph'].apply(is_logic_type)

# 3. 결과 집계
logic_count = test_df['is_logic'].sum()
total_count = len(test_df)
ratio = (logic_count / total_count) * 100

print(f"- 전체 데이터 개수: {total_count}개")
print(f"- 로직 타입으로 판정된 개수: {logic_count}개")
print(f"- 비중: {ratio:.2f}%")

- 전체 데이터 개수: 869개
- 로직 타입으로 판정된 개수: 133개
- 비중: 15.30%


In [16]:
SYSTEM_PROMPT ="""당신은 정확하고 객관적인 분석 능력을 갖춘 고도로 훈련된 '논리 분석가(Logic Analyst)'입니다.
당신의 임무는 주어진 [질문]과 관련 정보를 분석하여 인과 관계, 경제학적 원리, 수치 계산, 또는 임상적 증거에 기반해 최고 수준의 논리적 정확성과 명확성을 확보하고, 가장 타당한 정답을 도출하는 것입니다.
어떠한 대화형 문구나 사족도 포함하지 말고, 오직 최종 정답만을 출력하십시오.
"""

In [17]:
USER_PROMPT = """지문:
{paragraph}


질문:
{question}


선택지:
{choices}


문제를 해결할 때는 다음 단계를 **반드시 순서대로** 따르고, 각 단계별로 명확한 설명을 다음 단계에 제공하십시오:

**1. 유형 식별**: 문제의 핵심 분야(예: 경제학(계산/이론), 심리학(진단), 통계/인구학)를 명확히 식별하십시오.
**2. 핵심 정보 추출**: 주어진 [질문]과 [보기]에서 분석에 필요한 모든 핵심 변수(예: 금리, 수요, 증상, 수치 데이터)와 조건(예: 상승/하락, ~가 발생했다면)을 빠짐없이 정확하게 나열하십시오.
**3. 관련 원리 및 공식 적용**: 식별된 유형에 따라 해당 문제에 적용되는 주요 법칙, 이론, 또는 공식(예: 수요-공급 곡선의 이동, GDP 공식, 심리 진단 기준)을 제시하고 그 내용을 간략히 설명하십시오.
**4. 단계별 논리적 추론**: 추출된 조건과 적용된 원리를 바탕으로, 최종 결과에 도달하는 논리적 과정을 2~4문장으로 간결하게 서술하십시오. 복잡한 추론의 경우 핵심적인 인과 관계를 명확히 제시하십시오 (예: 금리 인상 -> 투자 감소 -> 총수요 감소).
**5. 최종 결론 및 정답 도출**: 단계적 추론 결과를 [보기] 및 [선택지]와 면밀히 비교하여, 가장 논리적으로 타당한 1, 2, 3, 4 중 하나의 숫자를 최종 정답으로 명확히 선택하십시오.

"""

In [18]:
def format_choices(choices: List[str]) -> str:
        """
        리스트 형태의 선택지를 번호와 함께 문자열로 포맷팅합니다.
        
        Args:
            choices: 선택지 텍스트들이 담긴 리스트
            
        Returns:
            "1 - 선택지" 형태로 줄바꿈된 전체 선택지 문자열
        """
        return "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(choices)])

In [19]:
def process_hard(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 100,
    num_samples: int = 5,  # Self-Consistency 샘플링 횟수 추가
) -> Dict:

    choices_string = format_choices(row_dict['choices'])

    user_message = USER_PROMPT.format(
        paragraph=row_dict["paragraph"],
        question=row_dict["question"],
        choices=choices_string,
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message},
    ]
    
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enabenable_thinking=False,
    )
    
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)
    
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answer = extract_answer(full_text)
    
    return {
        "id": row_dict.get("id"),
        "answer": answer,
        "full_output": full_text
    }

In [20]:
def process_row(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 100,
    num_samples: int = 5,  # Self-Consistency 샘플링 횟수 추가
) -> Dict:

    output = builder.build_message(row_dict)
    messages = output["messages"]
    
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enabenable_thinking=False,
    )
    
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)
    
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answer = extract_answer(full_text)
    
    return {
        "id": row_dict.get("id"),
        "answer": answer,
        "full_output": full_text
    }

In [21]:
results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
    if (idx + 1) % 100 == 0:
        print(f"  [{idx+1}/{len(test_df)}]")
    
    row_dict = row.to_dict()
    if row_dict['is_logic']:
        print("어려운 문제 로직")
        result = process_hard(
            row_dict,
            builder,
            tokenizer,
            model,
            device=device,
            max_new_tokens=max_new_tokens
        )
    else:
        result = process_row(
            row_dict,
            builder,
            tokenizer,
            model,
            device=device,
            max_new_tokens=max_new_tokens
        )
    results.append(result)


Inference:  11%|█▏        | 99/869 [03:34<17:13,  1.34s/it]

  [100/869]


Inference:  13%|█▎        | 110/869 [03:49<19:42,  1.56s/it]

어려운 문제 로직


Inference:  13%|█▎        | 111/869 [03:52<25:16,  2.00s/it]

...

Inference:  99%|█████████▉| 862/869 [26:21<00:13,  1.91s/it]

어려운 문제 로직


Inference: 100%|█████████▉| 866/869 [26:28<00:05,  1.70s/it]

어려운 문제 로직


Inference: 100%|██████████| 869/869 [26:33<00:00,  1.83s/it]


In [22]:
results[:10]

[{'id': 'generation-for-nlp-0',
  'answer': '5',
  'full_output': 'system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\nuser\n### 지문\n사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 .   독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의  즐거움’이 있다. 독자는 독서를 통해 책과 소통하는 즐거움을 경험한다 .  독서는   필자와 간접적으로 대화하는 소통 행위이다 .  독자는 자신이 속한   사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는  사회나 시대를 경험한다.  직접 경험하지 못했던 다양한 삶을  필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를   바라볼 수 있다.  이때 같은 책을 읽은 독자라도 독자의 배경 지식이나 관점 등의 독자 요인,  읽기 환경이나 과제 등의 상황  요인이 다르므로,  필자가 보여 주는 세계를 그대로 수용하지  않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다 . 이러한 소통은 독자가 책의 내용에 대해 질문하고 답을  찾아내는 과정에서 가능해진다.  독자는 책에서 답을 찾는  질문 ,  독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다 .   전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고,   책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로  구성할 수도 있다.  또한 후자의 경우 책에는 없는 독자의  경험에서 답을 찾을 수 있다.  이런 질문들을 풍부히 생성 하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다 . 한편 독자는 ㉠다른 독자와 소통하는 즐거움 을 경험할 수도  있다 .  책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임 이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당 한다.  비

In [24]:
print(f"Saving predictions to {output_path}...")
predictions_df = pd.DataFrame(results)
predictions_df = predictions_df[["id", "answer"]]

Path(output_path).parent.mkdir(parents=True, exist_ok=True)
predictions_df.to_csv(output_path, index=False)

Saving predictions to ./predictions.csv...
